## Notebook to generate mutation dataset for everyday cumulative between time points

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime as dt
from scipy.signal import lfilter
import pickle as pickle
import sys

### JH Time Series Data

* Time series data is cumlative per column

In [2]:
date_start = '01/23/20'
date_up_to = '12/31/20'
jh_path = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/'
confirmed_global = 'time_series_covid19_confirmed_global.csv'
deaths_global = 'time_series_covid19_deaths_global.csv'

confirmed = pd.read_csv(jh_path+confirmed_global)
deaths = pd.read_csv(jh_path+deaths_global)

#Need to adjust country names to match those in CNCB
confirmed.loc[confirmed['Country/Region']=='US','Country/Region'] = 'United States'
confirmed.loc[confirmed['Country/Region']=='Congo (Kinshasa)','Country/Region'] = 'Democratic Republic of the Congo'
confirmed.loc[confirmed['Country/Region']=='Korea, South','Country/Region'] = 'South Korea'
confirmed.loc[confirmed['Country/Region']=='Czechia','Country/Region'] = 'Czech Republic'
confirmed.loc[confirmed['Country/Region']=='Burma','Country/Region'] = 'myanmar'
confirmed.loc[confirmed['Country/Region']=='Congo (Brazzaville)','Country/Region'] = 'republic of the congo'
confirmed.loc[confirmed['Country/Region']=="Cote d'Ivoire",'Country/Region'] = 'cotedivoire'

confirmed['Country/Region'] = confirmed['Country/Region'].str.lower()
confirmed.set_index('Country/Region',inplace=True)


deaths.loc[deaths['Country/Region']=='US','Country/Region'] = 'United States'
deaths.loc[deaths['Country/Region']=='Congo (Kinshasa)','Country/Region'] = 'Democratic Republic of the Congo'
deaths.loc[deaths['Country/Region']=='Korea, South','Country/Region'] = 'South Korea'
deaths.loc[deaths['Country/Region']=='Czechia','Country/Region'] = 'Czech Republic'
deaths.loc[deaths['Country/Region']=='Burma','Country/Region'] = 'myanmar'
deaths.loc[deaths['Country/Region']=='Congo (Brazzaville)','Country/Region'] = 'republic of the congo'
deaths.loc[deaths['Country/Region']=="Cote d'Ivoire",'Country/Region'] = 'cotedivoire'

deaths['Country/Region'] = deaths['Country/Region'].str.lower()
deaths.set_index('Country/Region',inplace=True)

### Need other data table for population number
* doesn't matter for date as long as all the countries are covered, only getting population number

In [3]:
date = '12-31-2020.csv'
jh_path = '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
jh_data = pd.read_csv(jh_path+date)

#drop rows that have nans for incidence rate
jh_data = jh_data[~jh_data['Incident_Rate'].isnull()]
jh_data = jh_data[jh_data['Incident_Rate']!=0]

#matching countries of jh data to that of cncb countries
jh_data.loc[jh_data['Country_Region']=='Taiwan*','Country_Region'] = 'Taiwan'
jh_data.loc[jh_data['Country_Region']=='US','Country_Region'] = 'United States'
jh_data.loc[jh_data['Country_Region']=='Korea, South','Country_Region'] = 'South Korea'
jh_data.loc[jh_data['Country_Region']=='Czechia','Country_Region'] = 'Czech Republic'
jh_data.loc[jh_data['Country_Region']=='Burma','Country_Region'] = 'myanmar'
jh_data.loc[jh_data['Country_Region']=='Congo (Kinshasa)','Country_Region'] = 'Democratic Republic of the Congo'
jh_data.loc[jh_data['Country_Region']=='Congo (Brazzaville)','Country_Region'] = 'republic of the congo'
jh_data.loc[jh_data['Country_Region']=="Cote d'Ivoire",'Country_Region'] = 'cotedivoire'

jh_data['Population'] = np.ceil(jh_data['Confirmed']/jh_data['Incident_Rate']*100000)
jh_data['Country_Region'] = jh_data['Country_Region'].str.lower()

### CNCB Genome Meta Data

In [4]:
#cncb meta, need to match countries
column_names = ['Virus Strain Name','Accession ID','Related ID','Nuc.Completeness','Sequence Quality','Host','Location','Sample Collection Date','Submitting Lab']
meta = pd.read_excel('../metadata1_5.xlsx',usecols=column_names,index_col=1)
meta.fillna(' ',inplace=True)

#Remove low quality and partial reads, pretty sure cncb does not run variant annotation for these anyways
meta = meta[meta['Sequence Quality']!='Low']
meta = meta[meta['Nuc.Completeness']!='Partial']

#set country column and lowercase
meta['Country'] = meta['Location'].str.split('/').str[0].str.strip()
meta['Country'] = meta['Country'].str.lower()

#Adjust country typos
meta.loc[meta['Country']=='\u200eromania','Country'] = 'romania'
meta.loc[meta['Country']=='viet nam','Country'] = 'vietnam'
meta.loc[meta['Country']=='czech repubic','Country'] = 'czech republic'
meta.loc[meta['Country']=='ivory coast','Country'] = 'cotedivoire'

# #Remove Crimea and Palestine
meta = meta[meta['Country']!='crimea']
meta = meta[meta['Country']!='palestine']

#Filter using date as well
meta = meta[meta['Sample Collection Date']!='2020-00-00'] #bad dates in there
meta.loc[:,'Sample Collection Date'] = pd.to_datetime(meta['Sample Collection Date'], yearfirst=True)
dt_date_up_to = dt.strptime(date_up_to, "%m/%d/%y")
dt_date_start = dt.strptime(date_start, "%m/%d/%y")
meta = meta[(meta.loc[:,'Sample Collection Date']<dt_date_up_to) & (meta.loc[:,'Sample Collection Date']>dt_date_start)]

print(meta.shape)

(203403, 9)


### If there is a missing country in set difference, there will be nan data in output

In [5]:
set(meta['Country'])-set(confirmed.index.str.lower())

set()

In [6]:
len(set(meta['Country']))

117

### Using Infections per 100k and Fatality percentage
* Create IR/FR for all days. Cumlative up to that day.

In [7]:
jh_daily_ir = pd.DataFrame(columns=pd.to_datetime(confirmed.columns[4::])) #start from the second date because we take difference between dates
jh_daily_fr = pd.DataFrame(columns=pd.to_datetime(confirmed.columns[4::]))
country_daily_cts = pd.DataFrame(columns=pd.to_datetime(confirmed.columns[4::]))
country_death_cts = pd.DataFrame(columns=pd.to_datetime(confirmed.columns[4::]))

jh_country_lvl = pd.DataFrame(columns=['cases','deaths','population'])

for country in set(meta['Country'].str.lower()):
    population = jh_data.loc[jh_data['Country_Region']==country,'Population'].sum()
    summed_cases = confirmed.loc[confirmed.index==country,date_up_to].sum()
    summed_deaths = deaths.loc[deaths.index==country,date_up_to].sum()
    jh_country_lvl.loc[country] = [summed_cases,summed_deaths,population]
    
    cases_timeperiod = 1+confirmed.loc[confirmed.index==country].iloc[:,4::].sum().values
    deaths_timeperiod = deaths.loc[deaths.index==country].iloc[:,4::].sum().values
    jh_daily_ir.loc[country] = cases_timeperiod / population * 100000
    jh_daily_fr.loc[country] = np.nan_to_num(deaths_timeperiod / cases_timeperiod) * 100
    country_daily_cts.loc[country] = (confirmed.loc[[country]].iloc[:,4::].sum().values-confirmed.loc[[country]].iloc[:,3:-1].sum().values).clip(min=0)
    country_death_cts.loc[country] = (deaths.loc[[country]].iloc[:,4::].sum().values-deaths.loc[[country]].iloc[:,3:-1].sum().values).clip(min=0)

jh_country_lvl.sort_index(inplace=True)
jh_daily_ir.sort_index(inplace=True)
jh_daily_fr.sort_index(inplace=True)

In [8]:
jh_daily_ir.to_csv(f"temporal_files/daily_counts/daily_ir_{dt.today().strftime('%m_%d_%y')}.csv")
jh_daily_fr.to_csv(f"temporal_files/daily_counts/daily_fr_{dt.today().strftime('%m_%d_%y')}.csv")
country_daily_cts.to_csv(f"temporal_files/daily_counts/daily_cases_{dt.today().strftime('%m_%d_%y')}.csv")
country_death_cts.to_csv(f"temporal_files/daily_counts/daily_deaths_{dt.today().strftime('%m_%d_%y')}.csv")

### Parsing gff3 files
* Change to only parse files that aren't in collection of variants. Load a saved file with collect values and which gff3 files, only update.

In [13]:
date = '01_07_21'
with open(f"jh_cncb_daily_dates_{date}.pkl",'rb') as file: #Contains only country name for each date
    aggregated_mutations = pickle.load(file)
    
with open(f"jh_cncb_daily_dates_identifiers_{date}.pkl",'rb') as file:
    processed_identifiers = pickle.load(file)

In [14]:
all_names = os.listdir('../gff3_cncb_restructured')
column_names = ['variant type','start','end', 'info']
missing = []
date_range = pd.date_range(start=jh_daily_ir.columns[0], end=jh_daily_ir.columns[-1])

#Check if there is a processesd_identifier list, contains genome names already processed
if 'processed_identifiers' not in locals():
    processed_identifiers = []
    
#aggregated mutations, dictionary for each date, each sub dictionary contains a 3d list structure. [[],[],[]]
if 'aggregated_mutations' not in locals():
    #this is for country ir and fr for that specific day
    aggregated_mutations = {date:{} for date in date_range}
else:
    for date in date_range:
        if date not in aggregated_mutations.keys():
            aggregated_mutations[date] = {}

#for identifier in tqdm(meta[meta['Country']=='United States'].index):
for identifier in tqdm(meta.index):
    
    #Use prexisting list
    if identifier in processed_identifiers:
        continue

    #Searching for correct identifier
    #--------------------------
    #No alternate name is ' '
    file_name = ''
    #Check if accession id in file names, if not check related ids
    if '2019-nCoV_'+identifier+'_variants.gff3' in all_names:
        file_name = '2019-nCoV_'+identifier+'_variants.gff3'
    # checking alternate names
    elif meta.loc[identifier,'Related ID'] != ' ':
        for alt_identifier in meta.loc[identifier,'Related ID'].replace(' ','').split(','):
            if '2019-nCoV_'+alt_identifier+'_variants.gff3' in all_names:
                file_name = '2019-nCoV_'+alt_identifier+'_variants.gff3'
                break
        #Added in case alternate names are also not found in gffs
        if file_name == '':
            missing.append(identifier)
            continue
    # If file name has not been updated, then there is no matching identifier, move to next index
    elif file_name == '':
        missing.append(identifier)
        continue
    #--------------------------
    
    #Filtering files with no variants
    #--------------------------
    with open(f'../gff3_cncb_restructured/{file_name}') as text_file:
        lines = text_file.readlines()
        counter = 0
        for l in lines:
            if '#' in l:
                counter += 1
    #Number of info lines should be less than total, if not then there are no mutations
    #--------------------------
    
    #List to keep track of which files used already, save and import this in future to avoid redundant search
    processed_identifiers.append(identifier)
    
    if counter<len(lines) and meta.loc[identifier,'Country'] in set(jh_data['Country_Region']): #country needs to be found in jh data for inf/fata rates
        
        gff = pd.read_csv(f'../gff3_cncb_restructured/{file_name}',sep='\t',skiprows=counter,usecols=[1,3,4,8],names=column_names)
        info_df = pd.DataFrame(gff['info'].str.split(';').values.tolist(),columns=[0,1,'Ref','Alt','Description']).drop([0,1],axis=1)
        gff = gff.drop(['info'],axis=1)
        gff['Country'] = [meta.loc[identifier,'Country']]*gff.shape[0]
        temp_df = pd.concat([gff,info_df],axis=1)
        
        #Filtering alternate amino acid and reference for missense_variant and synonymous_variant
        missenses_ref = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str[-3]
        synonymous_ref = temp_df.loc[temp_df['Description'].str.contains('synonymous_variant'),'Description'].str.split(',').str[1].str[-1]
        temp_df['Ref_AA'] = pd.concat([missenses_ref,synonymous_ref])
        temp_df['Alt_AA'] = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str[-1]

        missenses_str = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str.split('.').str[-1]
        synonymous_str = temp_df.loc[temp_df['Description'].str.contains('synonymous_variant'),'Description'].str.split(',').str[1].str.split('.').str[-1]
        temp_df['AA'] = pd.concat([missenses_str,synonymous_str])
        
        temp_df.fillna('', inplace=True)
        temp_df['descriptor'] = temp_df['start'].astype(str)+','+temp_df['end'].astype(str)+','+temp_df['Ref']+','+temp_df['Alt']+\
        ','+temp_df['Description'].str.split(',').str[0].str.split('=').str[1]+','+temp_df['variant type']+','+temp_df['Ref_AA']+','+temp_df['Alt_AA']+\
        ','+temp_df['AA']

#         if any(temp_df['Ref'].str.contains('REF=A') & temp_df['Alt'].str.contains('ALT=GGTTC')):
#             break
        
    
        for var in temp_df['descriptor']:
            #Dictionary for each sample date, each df then has its own variant descriptor (duplicates across dates)
            if var not in aggregated_mutations[meta.loc[identifier,'Sample Collection Date']].keys():
                aggregated_mutations[meta.loc[identifier,'Sample Collection Date']][var] = []

            #Use country and sample collection date to parse jh_daily_ir/jh_daily_fr
            #Country
            aggregated_mutations[meta.loc[identifier,'Sample Collection Date']][var].append(temp_df.loc[0,'Country'])

print(len(missing))

  1%|▏         | 3030/203403 [00:00<00:12, 15642.81it/s]

not processed yet


  9%|▉         | 18166/203403 [00:03<01:11, 2600.66it/s]

not processed yet


 10%|█         | 20617/203403 [00:04<01:23, 2196.16it/s]

not processed yet


 11%|█         | 21497/203403 [00:05<01:27, 2088.55it/s]

not processed yet


100%|██████████| 203403/203403 [08:13<00:00, 412.33it/s]

4


In [ ]:
with open(f"jh_cncb_daily_dates_{dt.today().strftime('%m_%d_%y')}.pkl",'wb') as file:
    pickle.dump(aggregated_mutations,file,protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f"jh_cncb_daily_dates_identifiers_{dt.today().strftime('%m_%d_%y')}.pkl",'wb') as file:
    pickle.dump(processed_identifiers,file,protocol=pickle.HIGHEST_PROTOCOL)
    

### Need to combine sequences and their matched ir/fr for a sliding window, need to use same ir/fr for the whole window

* aggregated_muations dicitonary of dictionaries
* level 1 - dates
* level 2 - mutation names
* for each mutation on level 2 contains countries

In [ ]:
date_range = jh_daily_ir.columns[9::] #skip the first 9 (feb 1), arbitrarily since not too much data yet
aggregated_mutations_window = {date:{} for date in date_range}

#Gets cumlative country counts up to date

#iterate over dates to get cumlative up to that date
for date in tqdm(date_range):
    #iterate over the preceding dates for the day that you are calculating
    for window_date in pd.date_range(start=date_range[0],end=date):
        for var in aggregated_mutations[window_date].keys():
            if var not in aggregated_mutations_window[date].keys():
                aggregated_mutations_window[date][var] = [[],[],[]]
                
            #Use country and date to get same ir/fr for that period
            aggregated_mutations_window[date][var][0] += aggregated_mutations[window_date][var] #Appending country names
            
#             for country in aggregated_mutations[window_date][var]:
#                 aggregated_mutations_window[date][var][1].append(jh_daily_ir.loc[country,date])
#                 aggregated_mutations_window[date][var][2].append(jh_daily_fr.loc[country,date])


### Using cumlative country counts up to date
* Counts cumlative countires for a variant and multiplies counts by the cumlative ir/fr
* ir/fr is cumlative up to that date

In [ ]:
#Looping over single days
for date in tqdm(aggregated_mutations_window.keys()):
    #Looping over single variants
    for var in aggregated_mutations_window[date].keys():
        #For each variant count countries, 
        for counted in Counter(aggregated_mutations_window[date][var][0]).items(): #counted is country and counts
            aggregated_mutations_window[date][var][1].append(counted[1]*jh_daily_ir.loc[counted[0],date])
            aggregated_mutations_window[date][var][2].append(counted[1]*jh_daily_fr.loc[counted[0],date])

In [ ]:
#I dont think it finishes the last few dates, there are no sequences yet
for date in tqdm(aggregated_mutations_window.keys()):
    unique_muts = pd.DataFrame(list(aggregated_mutations_window[date].keys()),columns=['descriptor'])
    unique_muts = pd.DataFrame.join(unique_muts,pd.DataFrame(unique_muts['descriptor'].str.split(',').to_list())) #assign columns with parsed descriptor
    unique_muts.set_index('descriptor',inplace=True)
    unique_muts[0] = pd.to_numeric(unique_muts[0])
    unique_muts[1] = pd.to_numeric(unique_muts[1])
    unique_muts.sort_values([0,1],inplace=True)
    unique_muts.columns = ['Start','End','Ref','Alt','VEP','Variant Type','Ref_AA','Alt_AA','AA']

    counts = {}
    num_countries = {}
    infection_rate = {}
    fatality_rate = {}
    counted_countries = {}

    for desc in unique_muts.index:
        counts[desc] = len(aggregated_mutations_window[date][desc][0])
        num_countries[desc] = len(set(aggregated_mutations_window[date][desc][0]))
        infection_rate[desc] = sum(aggregated_mutations_window[date][desc][1])/counts[desc]
        #infection_rate[desc] = np.mean(aggregated_mutations_window[date][desc][1]) #since accumulated over countries before denomenator of mean is wrong
        fatality_rate[desc] = sum(aggregated_mutations_window[date][desc][2])/counts[desc]
        #fatality_rate[desc] = np.mean(aggregated_mutations_window[date][desc][2]) #since accumulated over countries before denomenator of mean is wrong
        counted_countries[desc] = dict(Counter(aggregated_mutations_window[date][desc][0]))

    unique_muts['counts'] = unique_muts.index.to_series().map(counts)
    unique_muts['countries'] = unique_muts.index.to_series().map(num_countries)
    unique_muts['infection_rate'] = unique_muts.index.to_series().map(infection_rate)
    unique_muts['fatality_rate'] = unique_muts.index.to_series().map(fatality_rate)
    unique_muts['counted_countries'] = unique_muts.index.to_series().map(counted_countries)
    unique_muts.index = unique_muts.index.str.split(',').str[0:4].str.join('_')
    unique_muts.sort_values('counts',ascending=False)
    unique_muts.to_csv(f"temporal_files/cumulative_daily1_7/{date.strftime('%m_%d_%y')}.csv")